[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jrkasprzyk/CVEN5393/blob/main/Colab%20Notebooks/mass-curve-performance_workbook.ipynb)

*This notebook is part of course notes for CVEN 5393: Water Resource Systems and Management, by Prof. Joseph Kasprzyk at CU Boulder.*

This notebook reflected an interactive class discussion that implemented the  Loucks and van Beek example of Sequent Peak. Currently it's not well explained, because it was just written as we went in a class period.

In [ ]:
import pandas as pd #for dataframes and data processing
import numpy as np #for numerical computation
import matplotlib.pyplot as plt #for plotting
import sys #system functions
from scipy import interpolate #bring in only the interpolate function

In [ ]:
inflow = [1.0, 3.0, 3.0, 5.0, 8.0, 6.0, 7.0, 2.0, 1.0]
inflow_doubled = np.concatenate((inflow, inflow), axis=None)
example_data = pd.DataFrame(inflow_doubled, columns=["Inflow"])
example_data["Demand"] = 3.5

example_data

,Inflow,Demand
0,1.0,3.5
1,3.0,3.5
2,3.0,3.5
3,5.0,3.5
4,8.0,3.5
5,6.0,3.5
6,7.0,3.5
7,2.0,3.5
8,1.0,3.5
9,1.0,3.5


In [ ]:
def find_capacity(df):
  # input: input_data is DataFrame with Inflow and Demand
  # returns capacity using Sequent Peak method

  R = df["Demand"].to_numpy(copy=True)
  Q = df["Inflow"].to_numpy(copy=True)
  N = len(R)
  K = np.zeros(N)

  K[0] = 0.0
  for i in range(1, N):
    if R[i]-Q[i]+K[i-1]>0.0:
      K[i] = R[i]-Q[i]+K[i-1]
    else:
      K[i] = 0.0

  return np.max(K)

In [ ]:
capacity = find_capacity(example_data)

In [ ]:
capacity

7.5

In [ ]:
def res_sim_basic(input_data, initial_storage, storage_capacity):

  # adapted from S. Galelli MATLAB reservoir simulation code

  # input:
  # input_data -- data frame with Inflow and Demand for the entire timeseries,
  # in volume units.
  #
  # initial_storage -- initial storage in volume units
  #
  # storage_capacity -- storage capacity in volume units (any storage
  # greater than this volume will spill)

  # output:
  # a dataframe with:
  # Storage, Delivery, Spill, Outflow

  # Assumptions: assume reservoir can release all storage to meet demand
  # spill occurs if storage exceeds capacity
  # no capacity constraints for releasing spill

  # CORRECTION: copy=True is necessary so that the dataframe doesn't
  # improperly reference itself to the original data
  inflow = input_data['Inflow'].to_numpy(copy=True)
  demand = input_data['Demand'].to_numpy(copy=True)

  N = len(inflow)

  # create new arrays to store variables, each the same length
  storage = np.zeros(N)
  delivery = np.zeros(N)
  spill = np.zeros(N)
  outflow = np.zeros(N)

  storage[0] = initial_storage

  for i in range(N):
    # assume the inflow and the current storage volume at the current timestep
    # is all available to meet demand
    available_water = inflow[i] + storage[i]

    if available_water > demand[i]:
      # deliver all the water that is requested
      delivery[i] = demand[i]
    else:
      # only deliver what you have
      delivery[i] = available_water

    # if all we do is allow inflow to come in, and release delivery[i],
    # see how much storage there will be left
    trial_storage = storage[i] + inflow[i] - delivery[i]

    # check if trial_storage exceeds capacity, if so, spill the
    # excess water
    if trial_storage > storage_capacity:
      spill[i] = trial_storage - storage_capacity
      if i < N-1:
        storage[i+1] = storage_capacity
    else:
      spill[i] = 0
      if i < N-1:
        storage[i+1] = trial_storage

    # note: in the above, we only store the 'next storage value' up to
    # N-1, because of zero indexing (i.e. the last entry in the array has
    # index N-1

    # store outflow as the sum of both spill and delivery
    outflow[i] = spill[i] + delivery[i]

  # before exiting, save the data into the dataframe, which is the only thing
  # returned
  df = pd.DataFrame()
  df['Storage'] = storage
  df['Delivery'] = delivery
  df['Spill'] = spill
  df['Outflow'] = outflow

  return df

In [ ]:
example_result = res_sim_basic(example_data, 7.5, 7.5)

In [ ]:
example_full = pd.concat([example_data, example_result], axis=1)

In [ ]:
example_full

,Inflow,Demand,Storage,Delivery,Spill,Outflow
0,1.0,3.5,7.5,3.5,0.0,3.5
1,3.0,3.5,5.0,3.5,0.0,3.5
2,3.0,3.5,4.5,3.5,0.0,3.5
3,5.0,3.5,4.0,3.5,0.0,3.5
4,8.0,3.5,5.5,3.5,2.5,6.0
5,6.0,3.5,7.5,3.5,2.5,6.0
6,7.0,3.5,7.5,3.5,3.5,7.0
7,2.0,3.5,7.5,3.5,0.0,3.5
8,1.0,3.5,6.0,3.5,0.0,3.5
9,1.0,3.5,3.5,3.5,0.0,3.5


In [ ]:
example_full.to_excel('example.xlsx')